# 处理文本数据
## 文本数据的常见任务
- 文档分类或时间序列分类：比如说识别文档的类别或作者
- 时间序列对比：比如说比较两个文档的相似度
- seq2seq：比如说文本自动生成文本，机器翻译
- 情感分析：比如说分析电影的评价是好评还是差评
- 时间序列预测：比如说根据最近的天气去预测未来的天气
## word2vec
- 与计算机视觉技术一样，模型只能去处理数字，所以我们首先要做的就是把文本转换成数字，即将文本向量化，通常有一下三种方法：
  - 将文本转换成单词，然后将每个单词向量化
  - 将文本转换成字符，然后将每个字符向量化
  - 提取单词或字符的n-gram特征，然后将每个n-gram向量化
- 由文本转换而成的char, word, n-gram称为token，这个过程称为分词-tokenization
- 组织token和vector的关系有两种方式：one-hot, embedding

## word级别的one-hot编码

In [10]:
import numpy as np
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
# 创建一个字典，用来记录单词和索引
token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
token_index

{'The': 1,
 'cat': 2,
 'sat': 3,
 'on': 4,
 'the': 5,
 'mat.': 6,
 'dog': 7,
 'ate': 8,
 'my': 9,
 'homework.': 10}

In [11]:
# 最大的单词数量，只对样本中的前max_length个单词进行one-hot编码
max_length = 10
# 创建一个3D张量，用来记录one-hot编码
results = np.zeros(shape=(len(samples), max_length, max(token_index.values()) + 1))
results.shape

(2, 10, 11)

In [12]:
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.
results

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0

## 字符级别的one-hot编码

In [ ]:
import string
samples = ['the cat sat on the mat.', 'the dog ate my homework.']
# 所有可打印的ASCII字符
characters = string.printable
token_index = dict(zip(characters, range(1, len(characters) + 1)))
# 将所有的字符做成一个字典
token_index

In [14]:
max_length=50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.
results

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [15]:
results.shape

(2, 50, 101)

## 使用keras内置函数实现单词级别的one-hot编码

In [29]:
from keras.preprocessing.text import Tokenizer
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
# 创建一个Tokenizer对象，使用空格作为分隔符，对大小写不敏感
# num_words是最大单词数量，即手动实现中的token_index.values()+1
# 手动实现one-hot最后转换的是一个3D张量，这里是一个2D张量，所以不需要考虑每个max_length这个超参
# 手动实现的one-hot每一个单词是一个list，这里每一个样本是是一个list，所以不需要考虑每个max_length这个超参
tokenizer = Tokenizer(num_words=10)
tokenizer.fit_on_texts(samples)

In [30]:
# 编码成序列
sequences = tokenizer.texts_to_sequences(samples)
sequences

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]

In [31]:
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
one_hot_results

array([[0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 1., 1., 1.]])

## 词嵌入-embedding
- one-hot编码的矩阵是稀疏的，embedding编码的矩阵是密集的
- one-hot是对数据简单统计后的编码，embedding这种编码方式需要从数据中学习得到